In [11]:
%cd ~/work/deeplearning/

/home/asaeed9/work/deeplearning


In [12]:
from theano.sandbox import cuda
cuda.use('gpu1')
%matplotlib inline
from __future__ import print_function, division
#path="../data/2cat/sample"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink
from keras.preprocessing import image, sequence
import os, sys, cv2
from shutil import copyfile, move
from random import shuffle

####
from keras.layers.convolutional import *
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [13]:
data_dir="/home/asaeed9/work/data/2cat"
path="/home/asaeed9/work/data/2cat/sample/"
results_path = "/home/asaeed9/work/data/2cat/sample/results"
test_path = path + '/test/' #We use all the test data
%cd ../data/2cat/train

/home/asaeed9/work/data/2cat/train


In [ ]:
#grey images - minimal image augmentation

    
    #clean previous data
#     os.chdir(path + 'test')
    #adjust_prev_data()
    
    #build validation set
#     g = glob('*.jpg')
#     shuf = np.random.permutation(g)
#     for i in range(validation): os.rename(shuf[i], '../valid/' + shuf[i]) 
    
#     g = glob('*.jpg')
#     shuf = np.random.permutation(g)
#     for i in range(train): os.rename(shuf[i], '../sample/train/' + shuf[i]) 
#     %mv ../sample/train/cat*.jpg ../sample/train/cats/
#     %mv ../sample/train/dog*.jpg ../sample/train/dogs/
    
#     %cd ../valid

#     g = glob('*.jpg')
#     shuf = np.random.permutation(g)
#     for i in range(validation): os.rename(shuf[i], '../sample/valid/' + shuf[i]) 
#     %mv ../sample/valid/cat*.jpg ../sample/valid/cats/
#     %mv ../sample/valid/dog*.jpg ../sample/valid/dogs/
#     %cd $data_dir/train  


def adjust_prev_data_sample():
    %mv $path/train/cats/* $data_dir/train/
    %mv $path/train/dogs/* $data_dir/train/
    %mv $path/valid/cats/* $data_dir/train/
    %mv $path/valid/dogs/* $data_dir/train/

#clean previous data
def adjust_prev_data():
    %mv $data_dir/valid/* $data_dir/train/    
    adjust_prev_data_sample()

#move training images
def move_from_test(new_trainset, path, train, validation):
    print(new_trainset[:10])
      
    
#copy training images
def copy_samples(train, validation):
    #print("Copying new samples for training...")
    #clean previous data
    adjust_prev_data()
    
    #build validation set
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(validation): os.rename(shuf[i], '../valid/' + shuf[i]) 
    
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(train): copyfile(shuf[i], '../sample/train/' + shuf[i]) 
    %mv ../sample/train/cat*.jpg ../sample/train/cats/
    %mv ../sample/train/dog*.jpg ../sample/train/dogs/
    
    %cd ../valid

    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(validation): copyfile(shuf[i], '../sample/valid/' + shuf[i]) 
    %mv ../sample/valid/cat*.jpg ../sample/valid/cats/
    %mv ../sample/valid/dog*.jpg ../sample/valid/dogs/
    %cd $data_dir/train

def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

def pred_batch(imgs, classes):
    preds = model.predict(imgs)
    idxs = np.argmax(preds, axis=1)

    print('Shape: {}'.format(preds.shape))
    print('First 5 classes: {}'.format(classes[:5]))
    print('First 5 probabilities: {}\n'.format(preds[:5]))
    print('Predictions prob/class: ')
    
    for i in range(len(idxs)):
        idx = idxs[i]
        print ('  {:.4f}/{}'.format(preds[i, idx], classes[idx]))


def generate_size_graph(fig_no, training_size, accuracy, loss, start_size, end_size):
    plt.figure(fig_no,figsize=(7,5))
    plt.plot(training_size,accuracy)
    plt.plot(training_size,loss)
    plt.xlabel('Training Size')
    plt.ylabel('Accuracy/Loss')
    plt.title('Training Size vs Accuracy/Loss')
    plt.grid(True)
    plt.legend(['Accuracy','Loss'])
    plt.style.use(['classic'])
    plt.show()
    plt.savefig(path + '/batch_graphs/' +  str(start_size) + '_' + str(end_size) + '.jpg') 
        
def generate_graph(fig_no, epochs, train, val, label, train_title, val_title, train_size):
    plt.figure(fig_no,figsize=(7,5))
    plt.plot(epochs,train)
    plt.plot(epochs,val)
    plt.xlabel('num of Epochs')
    plt.ylabel(label)
    plt.title(train_title + ' vs ' + val_title + '( Samples:' + str(train_size) + ')')
    plt.grid(True)
    plt.legend(['train','val'])
    plt.style.use(['classic'])
    plt.show()
    plt.savefig(results_path + '/batch_graphs/' +  label + '_' + str(train_size) + '.jpg') 
    
def get_train_model(tr_batches, val_batches, epoch):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3, 256,256)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
#             Convolution2D(64,3,3, activation='relu'),
#             BatchNormalization(axis=1),
            #MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Flatten(),
            Dense(1024, activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            Dense(2, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=epoch - 2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model 

def train_model(model, tr_batches, val_batches, epoch):
    if not model:
        model = Sequential([
                BatchNormalization(axis=1, input_shape=(3, 256,256)),
                Convolution2D(32,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Dropout(0.2),
    #             Convolution2D(64,3,3, activation='relu'),
    #             BatchNormalization(axis=1),
                #MaxPooling2D((3,3)),
                Convolution2D(64,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Dropout(0.2),
                Convolution2D(32,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Dropout(0.2),
                Flatten(),
                Dense(1024, activation='relu'),
                BatchNormalization(),
                Dropout(0.2),
                Dense(2, activation='softmax')
            ])
        model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
       
    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=epoch - 2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
        
    return model 
    
    
def get_test_model():
    model = Sequential([
                BatchNormalization(axis=1, input_shape=(3,256,256)),
                Convolution2D(32,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
    #             Convolution2D(64,3,3, activation='relu'),
    #             BatchNormalization(axis=1),
                #MaxPooling2D((3,3)),
                Convolution2D(64,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Convolution2D(32,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Flatten(),
                Dense(1024, activation='relu'),
                BatchNormalization(),
                Dense(2, activation='softmax')
            ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])  
    
    return model

def fit(samples_copied, old_model, path, results_path, nepoch, batch_size, train_size, valid_size):
    gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

    #for train in training_range:
    model = None
    if not samples_copied:
        copy_samples(train_size, valid_size)

    tr_batches = gen_t.flow_from_directory(path + 'train', batch_size=batch_size)
    val_batches = gen_t.flow_from_directory(path + 'valid', class_mode='categorical', shuffle=True, batch_size=batch_size * 2)
    
    if old_model:
        model = train_model(old_model, tr_batches, val_batches, nepoch)
    else:
        model = train_model(None, tr_batches, val_batches, nepoch)
        
    model.save_weights(results_path+ '/' + 'ft_' + str(train_size) + '.e' + str(nepoch))
    
    return model

def predict(path, model):
    gen_test = image.ImageDataGenerator()
    test_batches = gen_test.flow_from_directory(path+'test', class_mode=None, target_size=(256,256), shuffle=False, batch_size=1)
    test_data = np.concatenate([test_batches.next() for i in range(test_batches.nb_sample)])
    test_labels = onehot(test_batches.classes)
    score = model.evaluate(test_data, test_labels)
    
    probs = model.predict(test_data)
    
    #loss_score.append(score[0])
    #accuracy_score.append(score[1])
    
    print("\nLoss:{}, Accuracy:{}".format(score[0], score[1]))
    #print("\nProbabilities:{}".format(probs))
    return probs, test_batches

def move_samples(retrain_set,dest_path, n, limit):
    cats_copied = 0
    dogs_copied = 0
    retrain_list = list(retrain_set)
    shuffle(retrain_list)
    for fil in range(n):
        fil = retrain_list.pop()
        fil_cpy = fil[fil.find('/')+1:]

        if 'cat' in fil_cpy and cats_copied <= limit:
            os.rename(os.path.join(path + "test/cats/"+ fil_cpy), os.path.join(path + dest_path + "/cats/"+ fil_cpy))
            cats_copied+=1
        elif 'dog' in fil_cpy and dogs_copied <= limit:
            os.rename(os.path.join(path + "test/dogs/"+ fil_cpy), os.path.join(path + dest_path + "/dogs/" + fil_cpy))
            dogs_copied+=1
            
    #print("Limit:", limit)        
    #print("moved cats:", cats_copied)
    #print("moved dogs:", dogs_copied)
    #print("Retrain Length: ", len(retrain_list))
    return retrain_list, cats_copied, dogs_copied


def move_to_train(retrain_set, limit):
    cats = 0
    dogs = 0
    valid_limit = int(math.floor(.2*(limit*2)))
    train_limit = int(math.floor(.8*(limit*2)))
    
    #print(retrain_set[:10])
    print("validation set: ", valid_limit)
    retrain_list, cats_copied, dogs_copied = move_samples(retrain_set, "valid", valid_limit, limit)
    valid_size = cats_copied + dogs_copied
    print("Train set: ", train_limit)
    retrain_list, cats_copied, dogs_copied = move_samples(retrain_list, "train", train_limit, limit)
    train_size = cats_copied + dogs_copied
    
    return train_size, valid_size, train_size + valid_size

def refil_test(nimages):
    os.chdir("/home/asaeed9/work/data/2cat/train")
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(nimages): os.rename(shuf[i], '../sample/test/' + shuf[i])
    #os.chdir("../sample/test/")
    #move("cat*.jpg")
    %mv ../sample/test/cat*.jpg ../sample/test/cats/
    %mv ../sample/test/dog*.jpg ../sample/test/dogs/     

In [ ]:
%cd $data_dir/train

nepoch = 15
batch_size = 32
train_size = 100
total_train_size = 100
valid_size = int(math.floor(.2 * train_size))
#print('sample size: {}'.format(train_size + valid_size))
tr_model = None

i=0
for i in range(20):
    #print("Train Size:{}".format(train_size))
    #print("Valid Size:{}".format(valid_size))
    tr_model = fit(i, tr_model, path, results_path, nepoch, batch_size, train_size, valid_size)

    model = None
    model = get_test_model() 
    model.load_weights(results_path+'/ft_' + str(train_size) + '.e' + str(nepoch))
    probs, test_batches = predict(path, model)

    retrain_idx = np.unique(np.where(np.logical_and(probs >=0.3, probs<=0.7))[0])
    retrain_set = [test_batches.filenames[i] for i in retrain_idx]

    os.chdir(path + 'test')
    ndog = sum('dog' in name for name in retrain_set)
    ncat =  sum('cat' in name for name in retrain_set)
    limit = min(ncat, ndog)

    # print("Total Retrain Images: {}".format(limit*2))

    #print('Images to be retrained:{}'.format(limit*2))
    #print('Dogs:{0}, Cats:{1}, Total:{2}'.format(ndog, ncat, ndog+ncat))

    # # # #remove all train/validation images
    # print("Total Retrain Images: {}".format(limit*2))
    #move existing training data to the store
    adjust_prev_data_sample()
    train_size, valid_size, copied_images = move_to_train(retrain_set, limit)
    total_train_size += train_size
    refil_test(copied_images)

print('Total No. of Images:{}'.format(total_train_size))

/home/asaeed9/work/data/2cat/train
/home/asaeed9/work/data/2cat/valid
/home/asaeed9/work/data/2cat/train
Found 100 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/2
100/100 [==============================] - 1s - loss: 0.9318 - acc: 0.5900 - val_loss: 0.7443 - val_acc: 0.5500
Epoch 2/2
100/100 [==============================] - 1s - loss: 0.9035 - acc: 0.5700 - val_loss: 0.6656 - val_acc: 0.6000
Epoch 1/13
100/100 [==============================] - 1s - loss: 0.6460 - acc: 0.7000 - val_loss: 0.6784 - val_acc: 0.6500
Epoch 2/13
100/100 [==============================] - 1s - loss: 0.6857 - acc: 0.6200 - val_loss: 0.6952 - val_acc: 0.6000
Epoch 3/13
100/100 [==============================] - 1s - loss: 0.6260 - acc: 0.7100 - val_loss: 0.7035 - val_acc: 0.4500
Epoch 4/13
100/100 [==============================] - 1s - loss: 0.8512 - acc: 0.6700 - val_loss: 0.7018 - val_acc: 0.5500
Epoch 5/13
100/100 [==============================] - 1s - loss: 0.5858 - acc:

319/319 [==============================] - 3s - loss: 0.6750 - acc: 0.7179 - val_loss: 0.7721 - val_acc: 0.4706
Epoch 13/13
319/319 [==============================] - 4s - loss: 0.6735 - acc: 0.6928 - val_loss: 0.8550 - val_acc: 0.4588
Found 3176 images belonging to 2 classes.
3168/3176 [============================>.] - ETA: 0s
Loss:0.757014365881, Accuracy:0.549748110831
validation set:  326
Train set:  1305
Found 1305 images belonging to 2 classes.
Found 326 images belonging to 2 classes.
Epoch 1/2
1305/1305 [==============================] - 20s - loss: 1.0313 - acc: 0.5448 - val_loss: 0.7152 - val_acc: 0.5000
Epoch 2/2
1305/1305 [==============================] - 16s - loss: 0.8903 - acc: 0.6061 - val_loss: 1.2893 - val_acc: 0.4693
Epoch 1/13
1305/1305 [==============================] - 19s - loss: 0.8095 - acc: 0.6161 - val_loss: 2.2763 - val_acc: 0.4387
Epoch 2/13
1305/1305 [==============================] - 16s - loss: 0.7810 - acc: 0.6521 - val_loss: 2.3607 - val_acc: 0.4571
E

592/592 [==============================] - 6s - loss: 0.7456 - acc: 0.6520 - val_loss: 0.8846 - val_acc: 0.5135
Epoch 10/13
592/592 [==============================] - 6s - loss: 0.7533 - acc: 0.6672 - val_loss: 0.8341 - val_acc: 0.5068
Epoch 11/13
592/592 [==============================] - 7s - loss: 0.7024 - acc: 0.6453 - val_loss: 0.8796 - val_acc: 0.5270
Epoch 12/13
592/592 [==============================] - 6s - loss: 0.7118 - acc: 0.6622 - val_loss: 0.7695 - val_acc: 0.5676
Epoch 13/13
592/592 [==============================] - 7s - loss: 0.7030 - acc: 0.6723 - val_loss: 0.7958 - val_acc: 0.6216
Found 3176 images belonging to 2 classes.
3168/3176 [============================>.] - ETA: 0s
Loss:0.792986744597, Accuracy:0.563916876574
validation set:  268
Train set:  1075
Found 1075 images belonging to 2 classes.
Found 268 images belonging to 2 classes.
Epoch 1/2
1075/1075 [==============================] - 16s - loss: 0.9345 - acc: 0.5777 - val_loss: 0.7413 - val_acc: 0.5299
Epoch 

In [ ]:








# cats_copied=5
# dogs_copied=5
# for fil in retrain_set:
#     #print("Retrain File: {}".format(fil))
#     fil_cpy = fil[fil.find('/')+1:]
#     if 'cat' in fil and cats_copied <= to_be_copied:
#         #print(os.path.join(path + "test/cats/"+ fil_cpy))
#         #print(os.path.join(path + "test_copy/"+ fil_cpy))
#         copyfile(os.path.join(path + "test/cats/"+ fil_cpy), os.path.join(path + "test_copy/"+ fil_cpy))
#         cats_copied+=1
#     elif 'dog' in fil and dogs_copied <= to_be_copied:
#         #print(os.path.join(path + "test/dogs/"+ fil_cpy))
#         #print(os.path.join(path + "test_copy/"+ fil_cpy))              
#         copyfile(os.path.join(path + "test/dogs/"+ fil_cpy), os.path.join(path + "test_copy/"+ fil_cpy))
#         dogs_copied+=1
    
#     if cats_copied >= to_be_copied and dogs_copied >= to_be_copied:
#         #print(len(os.listdir(path + "test_copy/")))
#         print(cats_copied, dogs_copied, to_be_copied)
#         break

# print("copy done.")

#move to validation directory
#replenish validation set

#move_to_test(cats_copied+dogs_copied)



        
        
        
        
        
        
        
        
    

    
#     for fil in retrain_set:
#         fil_cpy = fil[fil.find('/')+1:]
#         if 'cat' in fil and cats_copied <= limit:
#             #print(os.path.join(path + "test/cats/"+ fil_cpy))
#             #print(os.path.join(path + "test_copy/"+ fil_cpy))
#             copyfile(os.path.join(path + "test/cats/"+ fil_cpy), os.path.join(path + "test_copy/"+ fil_cpy))
#             cats_copied+=1
#         elif 'dog' in fil and dogs_copied <= limit:
#             #print(os.path.join(path + "test/dogs/"+ fil_cpy))
#             #print(os.path.join(path + "test_copy/"+ fil_cpy))              
#             copyfile(os.path.join(path + "test/dogs/"+ fil_cpy), os.path.join(path + "test_copy/"+ fil_cpy))
#             dogs_copied+=1

#         if cats_copied >= limit and dogs_copied >= limit:
#             #print(len(os.listdir(path + "test_copy/")))
#             print(cats_copied, dogs_copied, limit)
#             break

    
    
    

# def restore_backup(source, destination):
#     %mv $source/cat*.jpg $destination/cats/
#     %mv $source/dog*.jpg $destination/dogs/
    
# def remove_to_backup(source, destination):
#     %mv $source/* $destination/

# remove_to_backup(path + 'train/cats/', path + 'train_bkup/')
# remove_to_backup(path + 'train/dogs/', path + 'train_bkup/')

#restore_backup(path + 'train_bkup', path + 'train')

# nimages = cats_copied + dogs_copied
# def move_to_test(nimages):
#     os.chdir("/home/asaeed9/work/data/2cat/train")
#     g = glob('*.jpg')
#     shuf = np.random.permutation(g)
#     for i in range(nimages): os.rename(shuf[i], '../sample/test/' + shuf[i])
#     os.chdir("../sample/test/")
#     move("cat*.jpg")
#     %mv ../sample/test/cat*.jpg ../sample/cats/
#     %mv ../sample/test/dog*.jpg ../sample/dogs/    




#move_from_test(retrain_set, path, train_size, validation_size)


In [ ]:
model = None
model = get_test_model()
loss_score = []
accuracy_score = []
training_size = [100]

os.chdir(results_path)
files = filter(os.path.isfile, os.listdir(results_path))
files.sort(key=lambda x: os.path.getmtime(x))

for f in files:
    
    print("\nFile being processed: {}".format(f))
    
    train_size = int(f[f.find('_')+1 : f.find('.')])
    epoch = int(f[f.find('e')+1 : ])

    model.load_weights(results_path+'/ft_' + str(train_size) + '.e' + str(epoch))
    
    gen_test = image.ImageDataGenerator()
    test_batches = gen_test.flow_from_directory(path+'test', class_mode=None, target_size=(256,256), shuffle=False, batch_size=1)
    test_data = np.concatenate([test_batches.next() for i in range(test_batches.nb_sample)])
    test_labels = onehot(test_batches.classes)
    score = model.evaluate(test_data, test_labels)
    
    loss_score.append(score[0])
    accuracy_score.append(score[1])
    
    print("\nLoss:{}, Accuracy:{}".format(score[0], score[1]))


In [ ]:
model = None
model = get_test_model()
loss_score = []
accuracy_score = []
training_size = [100]

os.chdir(results_path)
files = filter(os.path.isfile, os.listdir(results_path))
files.sort(key=lambda x: os.path.getmtime(x))

for f in files:
    
    print("\nFile being processed: {}".format(f))
    
    train_size = int(f[f.find('_')+1 : f.find('.')])
    epoch = int(f[f.find('e')+1 : ])

    model.load_weights(results_path+'/ft_' + str(train_size) + '.e' + str(epoch))
    
    gen_test = image.ImageDataGenerator()
    test_batches = gen_test.flow_from_directory(path+'test', class_mode=None, target_size=(256,256), shuffle=False, batch_size=1)
    test_data = np.concatenate([test_batches.next() for i in range(test_batches.nb_sample)])
    test_labels = onehot(test_batches.classes)
    score = model.evaluate(test_data, test_labels)
    
    probs = model.predict(test_data)
    
    loss_score.append(score[0])
    accuracy_score.append(score[1])
    
    print("\nLoss:{}, Accuracy:{}".format(score[0], score[1]))
    print("\nProbabilities:{}".format(probs))


In [ ]:
test_path

In [ ]:
from PIL import Image
Image.open(test_path +  'cats/cat.6013.jpg')


In [ ]:
#probs[:10]
retrain_idx = np.where(np.logical_and(probs >=0.4, probs<=0.6))[0]
retrain_set = [test_batches.filenames[i] for i in retrain_idx]



retrain_set[:10]

In [ ]:
def get_train_model(tr_batches, epoch):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3, 256,256)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
#             Convolution2D(64,3,3, activation='relu'),
#             BatchNormalization(axis=1),
            #MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Flatten(),
            Dense(1024, activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            Dense(2, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    #model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=2)
    #model.optimizer.lr = 0.001
    #model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=epoch - 2)
    return model 

In [ ]:
#model = get_train_model(new_data, nepoch)
#model.load_weights(results_path + "/ft_100.e50")
#model.fit_generator(new_data, len(new_data), nb_epoch = 2)